In [1]:
!pip install openai=='0.28.0'

# Any user can either upvote or downvote a comment, the net value is the score of the comment.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

import openai

In [21]:
#Open AI set up

#4,oMWXRSsSu58dYT6KbF4R,50000000,Public Opinion
openai.api_base="http://91.107.239.71:80"
api_key = "oMWXRSsSu58dYT6KbF4R"
openai.api_key = api_key

In [3]:
#Call the json file with all the reddit comments for all topics

#with open('/content/drive/MyDrive/comments_for_prompts.json') as f:
#    reddit_comments = json.load(f)

with open('comments_for_prompts.json') as f:
    reddit_comments = json.load(f)

In [4]:
reddit_comments.keys()

dict_keys(['abortion', 'climate change', 'gun control', 'healthcare', 'immigration'])

In [5]:
reddit_comments['climate change']

["Headline: Los Angeles records county's highest temperature ever on record\n",
 "Comment 0: 'This may be the hottest summer in the last hundred years, but, on the bright side, it's the coldest summer of the next one hundred years!' (score: 5310)\n",
 "Comment 1: 'LA is so hot right now' (score: 3550)\n",
 "Comment 2: 'At 7:00 PM it was still 103 here in the LA basin.  I've lived here my whole life and I've never EVER seen it stay that hot that late into the afternoon.\r\n\r\nAt least it was a dry heat.' (score: 2464)\n",
 'Comment 3: \'Also of note "highest temperature ever reliably recorded on Earth" was recorded 2 weeks ago in Death Valley California on August 16th at 130 degrees Fahrenheit (54.4 celsius).\' (score: 1758)\n',
 "Comment 4: '121° that's just insane.' (score: 1082)\n",
 "Comment 5: 'That's about 50 degree Celsius for others trying to scroll down for a conversion' (score: 777)\n",
 "Comment 6: 'It was high 90’s in Colorado, this weekend. We’re getting 30° and snow on Tu

In [6]:
#Call anes survey questions and answer categories

#anes = pd.read_csv("/content/drive/MyDrive/anes_timeseries_2020_csv_20220210.csv")
anes = pd.read_csv("anes_timeseries_2020_csv_20220210.csv")

C:\Users\agust\AppData\Local\Temp\ipykernel_25716\3521680666.py:2: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types. Specify dtype option on import or set low_memory=False.
  anes = pd.read_csv("anes_timeseries_2020_csv_20220210.csv")


In [7]:
#Question code to topic
questcode_topic = {"V201336": "abortion",
              "V201342x": "abortion",
              "V202332": "climate change",
              "V202333": "climate change",
              "V202337": "gun control",
              "V202341x": "gun control",
              "V202344x": "gun control",
              "V202232": "immigration",
              "V202233": "immigration",
              "V202236x":"immigration",
              "V202380x": "public health"}

In [8]:
# Questions
questions = {
# Abortion 1:
"V201336":
"""There has been some discussion about abortion during recent
years. Which one of the opinions on this page best agrees with
your view?
1. By law, abortion should never be permitted
2. The law should permit abortion only in case of rape, incest, or
when the woman’s life is in danger
3. The law should permit abortion other than for
rape/incest/danger to woman but only after need clearly
established
4. By law, a woman should always be able to obtain an abortion
as a matter of personal choice"""
# Put "You can just tell me the number of the opinion you choose." away not to mix the LLMs


# Abortion 2:
, "V201342x":
"""Would you be pleased, upset, or neither pleased nor upset if the
Supreme Court reduced abortion rights?
1. Extremely pleased
2. Moderately pleased
3. A little pleased
4. Neither pleased nor upset
5. A little upset
6. Moderately upset
7. Extremely upset"""


# Climate change 1:
, "V202332":
"""How much, if at all, do you think climate change is currently
affecting severe weather events or temperature patterns in the
United States?
1. Not at all
2. A little
3. A moderate amount
4. A lot
5. A great deal"""

# Climate change 2:
, "V202333":
"""How important is the issue of climate change to you personally?
1. Not at all important
2. A little important
3. Moderately important
4. Very important
5. Extremely important"""


# Gun control 1:
, "V202337":
"""Do you think the federal government should make it more
difficult for people to buy a gun than it is now, make it easier for
people to buy a gun, or keep these rules about the same as they
are now?
1. More difficult
2. Easier
3. Keep these rules about the same"""

# Gun control 2:
, "V202341x":
"""Do you favor, oppose, or neither favor nor oppose requiring
background checks for gun purchases at gun shows or other
private sales?
1. Favor a great deal
2. Favor a moderate amount
3. Favor a little
4. Neither favor nor oppose
5. Oppose a little
6. Oppose a moderate amount
7. Oppose a great deal"""

# Gun control 3:
, "V202344x":
"""Do you favor, oppose, or neither favor nor oppose banning the
sale of semi-automatic “assault-style” rifles?
1. Favor a great deal
2. Favor a moderate amount
3. Favor a little
4. Neither favor nor oppose
5. Oppose a little
6. Oppose a moderate amount
7. Oppose a great deal"""

# Immigration 1:
, "V202232":
"""Do you think the number of immigrants from foreign countries
who are permitted to come to the United States to live should be
increased a lot, increased a little, left the same as it is now,
decreased a little, or decreased a lot?
1. Increased a lot
2. Increased a little
3. Left the same as it is now
4. Decreased a little
5. Decreased a lot"""

# Immigration 2:
, "V202233":
"""Now I’d like to ask you about immigration in recent years.
How likely is it that recent immigration levels will take jobs away
from people already here – extremely likely, very likely,
somewhat likely, or not at all likely?
1. Extremely likely
2. Very likely
3. Somewhat likely
4. Not at all likely"""

# Immigration 3:
, "V202236x":
"""Do you favor, oppose, or neither favor nor oppose allowing
refugees who are fleeing war, persecution, or natural disasters
in other countries to come to live in the U.S.?
1. Favor a great deal
2. Favor a moderate amount
3. Favor a little
4. Neither favor nor oppose
5. Oppose a little
6. Oppose a moderate amount
7. Oppose a great deal"""

# Public Health 1:
, "V202380x":
"""Do you favor an increase, decrease, or no change in
government spending tohelp people pay for health insurance
when they can’t pay for it all themselves?
1. Increase a great deal
2. Increase a moderate amount
3. Increase a little
4. No change
5. Decrease a little
6. Decrease a moderate amount
7. Decrease a great deal"""
}

In [9]:
def prompt_comments(comments, question, num_respondents):
    prompt_text = 'We well share you a headline and a number of comments on a topic on reddit from 2020. The comments score\
    represents the net value of upvotes and downvore. Afterwards you will be given a question followed several possible answer categories.\
    Comments: %s.   \
    Please estimate how a representative sample of %d US-Americans would have answered the following question in 2020. Provide a \
    frequency table that shows answer frequency of each answer option (output a dictionary object with answer category \
    number as key and answer frequency as value): %s'%(comments, num_respondents, question)

    return prompt_text

In [14]:
#Test with one question
question = "V201336"
topic = questcode_topic[question]

#original survey valid answers
valid_ans = anes[anes[question] > 0]
survey_ans = len(valid_ans)

#prompt for question (question + total answers)
prompt_text = prompt_comments(reddit_comments[topic], #comments on the topic
                              questions[question], #question with categories
                              survey_ans) #number of respondents

#Enough tokens
#https://platform.openai.com/docs/models/gpt-3-5-turbo
#https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
print("Aprox tokens:", (len(prompt_text.split()))*100/75)

#Answer generation
responses = openai.ChatCompletion.create(model="gpt-3.5-turbo-0125",
                                         messages=[{"role": "user", "content": prompt_text}]#,
                                         #n = 5
                                         )

final_ans = [i['message']['content'] for i in responses['choices']]
final_ans_dict = [eval(i) for i in final_ans]

#also include in the final object the info on the total number of respondents
final_ans_dict.append(survey_ans)
final_ans_dict

[{3: 5900, 2: 1220, 4: 850, 1: 238}, 8208]

In [22]:
#Model for mantaining session open and give input
message = {"role":"user", "content": input("This is the beginning of your chat with AI. [To exit, send \"###\".]\n\nYou:")};

conversation = [{"role": "system", "content": "Answer as straightforward as you can"}]

while(message["content"]!="###"):
    conversation.append(message)
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo-0125", messages=conversation)
    message["content"] = input(f"Assistant: {completion.choices[0].message.content} \nYou:")
    print()
    conversation.append(completion.choices[0].message)

In [27]:
#Model for mantaining session open for the project
prompt = prompt_text

# Split prompt into chunks of 4000 tokens
prompt_chunks = [prompt[i:i+4000] for i in range(0, len(prompt), 4000)]

message = {"role":"user", "content": prompt_chunks[0]}

conversation = [{"role": "system", "content": "Just mantain the session open and answer when the answer options are given to you"}]

for chunk in prompt_chunks[1:]:
    conversation.append(message)
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo-0125", messages=conversation)
    message["content"] = chunk
    print()
    conversation.append(completion.choices[0].message)

In [30]:
conversation

[{'role': 'system',
  'content': 'Just mantain the session open and answer when the answer options are given to you'},
 {'role': 'user',
  'content': ': -18)\\n\', "Comment 119: \'That’s promoting murder right there... smh\' (score: -18)\\n", "Comment 120: \'I don\'t get all the enthusiasm for killing another human being.\' (score: -19)\\n", "Comment 121: \'hooray!  more dead babies!\' (score: -19)\\n", "Comment 122: \'The most dangerous place for children is their mother\'s womb\' (score: -19)\\n", "Comment 123: \'Massmurder clinic? My favorite\' (score: -19)\\n", "Comment 124: \'I don\'t really think this is newsworthy\' (score: -19)\\n", "Comment 125: \'Oops you spelled George Soros wrong.\' (score: -20)\\n", "Comment 126: \'Guy ensures thousands of baby\'s can continue to be killed in Alabama\' (score: -22)\\n", "Comment 127: \'Gotta make sure those girls can have their cake and eat it too!\' (score: -22)\\n", "Comment 128: \'Burn it to the ground\' (score: -23)\\n", "Comment 129: 